In [85]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

# Wikipedia Page

In [86]:
#Import Data
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [87]:
#Filter Borough
df = df[df['Borough']!='Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [88]:
#Neighboorhood Grouping
df2 = df.groupby(['Postcode','Borough']).Neighbourhood.agg([('count', 'count'), ('Neighbourhood', ', '.join)])

df2['Postcode']= df2.index.get_level_values(level=0)
df2['Borough']= df2.index.get_level_values(level=1)

del df2['count']
del df

s = pd.Series(np.arange(0,len(df2)))
df2=df2.set_index(s)

df2.head()

,Neighbourhood,Postcode,Borough
0,"Rouge, Malvern",M1B,Scarborough
1,"Highland Creek, Rouge Hill, Port Union",M1C,Scarborough
2,"Guildwood, Morningside, West Hill",M1E,Scarborough
3,Woburn,M1G,Scarborough
4,Cedarbrae,M1H,Scarborough


In [89]:
#Import Town coordinates
Coor_Town = pd.read_csv('http://cocl.us/Geospatial_data')
Coor_Town.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [90]:
df3 = pd.merge(df2, Coor_Town, left_on='Postcode', right_on='Postal Code', how='left')
del df3['Postal Code']
del df2
del Coor_Town
df3.head()

,Neighbourhood,Postcode,Borough,Latitude,Longitude
0,"Rouge, Malvern",M1B,Scarborough,43.806686,-79.194353
1,"Highland Creek, Rouge Hill, Port Union",M1C,Scarborough,43.784535,-79.160497
2,"Guildwood, Morningside, West Hill",M1E,Scarborough,43.763573,-79.188711
3,Woburn,M1G,Scarborough,43.770992,-79.216917
4,Cedarbrae,M1H,Scarborough,43.773136,-79.239476


# Analysis using Foursquare

In [91]:
#Credential params
CLIENT_ID = '2XTXUDPQTGZFAHBYZ1UKI4PNDRO0WYTPVX2BJMYV32ANXOPX' # your Foursquare ID
CLIENT_SECRET = 'MKH4FVRUVQNDC3MAQE350E2ACWGRSL311B5YMMLTXLBQ5CIZ' # your Foursquare Secret
VERSION = '20200201'
LIMIT = 30

In [92]:
address = "Toronto, Canada"

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.653963 -79.387207


In [93]:
#Art related, within 3000 mts
search_query = 'Art'
radius = 3000

#Define URL, with params
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=2XTXUDPQTGZFAHBYZ1UKI4PNDRO0WYTPVX2BJMYV32ANXOPX&client_secret=MKH4FVRUVQNDC3MAQE350E2ACWGRSL311B5YMMLTXLBQ5CIZ&ll=43.653963,-79.387207&v=20200201&query=Art&radius=3000&limit=30'

In [94]:
#Get Json
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e59bfa70be7b4001b2ac7ea'},
 'response': {'venues': [{'id': '4ad4c05ef964a520daf620e3',
    'name': 'Art Gallery of Ontario',
    'location': {'address': '317 Dundas St W',
     'crossStreet': 'at Beverley St',
     'lat': 43.654002860337386,
     'lng': -79.39292172707437,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.654002860337386,
       'lng': -79.39292172707437}],
     'distance': 460,
     'postalCode': 'M5T 1G4',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['317 Dundas St W (at Beverley St)',
      'Toronto ON M5T 1G4',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1e2931735',
      'name': 'Art Gallery',
      'pluralName': 'Art Galleries',
      'shortName': 'Art Gallery',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/artgallery_',
       'suffix': '.png'},
      'primary': True}],
    'venuePage': {'id':

In [95]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,4ad4c05ef964a520daf620e3,Art Gallery of Ontario,"[{'id': '4bf58dd8d48988d1e2931735', 'name': 'Art Gallery', 'pluralName': 'Art Galleries', 'shortName': 'Art Gallery', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/artgallery_', 'suffix': '.png'}, 'primary': True}]",v-1582940072,False,317 Dundas St W,at Beverley St,43.654003,-79.392922,"[{'label': 'display', 'lat': 43.654002860337386, 'lng': -79.39292172707437}]",460,M5T 1G4,CA,Toronto,ON,Canada,"[317 Dundas St W (at Beverley St), Toronto ON M5T 1G4, Canada]",33853777
1,4ad4c064f964a52065f820e3,Ontario College of Art and Design University (OCADU),"[{'id': '4bf58dd8d48988d1ae941735', 'name': 'University', 'pluralName': 'Universities', 'shortName': 'University', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/education/default_', 'suffix': '.png'}, 'primary': True}]",v-1582940072,False,100 McCaul St,at Dundas St W,43.652803,-79.391074,"[{'label': 'display', 'lat': 43.65280251171013, 'lng': -79.3910743699992}]",337,M5T 1W1,CA,Toronto,ON,Canada,"[100 McCaul St (at Dundas St W), Toronto ON M5T 1W1, Canada]",NaN
2,4adf3c01f964a5208f7821e3,Aboveground Art Supplies,"[{'id': '4bf58dd8d48988d127951735', 'name': 'Arts & Crafts Store', 'pluralName': 'Arts & Crafts Stores', 'shortName': 'Arts & Crafts', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/artstore_', 'suffix': '.png'}, 'primary': True}]",v-1582940072,False,74 McCaul St,Dundas St.,43.652646,-79.390925,"[{'label': 'display', 'lat': 43.652645648070006, 'lng': -79.3909251764317}]",333,M5T 3K2,CA,Toronto,ON,Canada,"[74 McCaul St (Dundas St.), Toronto ON M5T 3K2, Canada]",NaN
3,4f64f2c0e4b0c7552d4dbfbf,CMU College of Makeup Art & Design,"[{'id': '4bf58dd8d48988d13b941735', 'name': 'School', 'pluralName': 'Schools', 'shortName': 'School', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_', 'suffix': '.png'}, 'primary': True}]",v-1582940072,False,110 Lombard St.,NaN,43.652516,-79.373252,"[{'label': 'display', 'lat': 43.65251631855119, 'lng': -79.37325153352174}]",1135,M5C 1M3,CA,Toronto,ON,Canada,"[110 Lombard St., Toronto ON M5C 1M3, Canada]",NaN
4,4f0b5bfae4b06fab1c0478f2,Department of Art,"[{'id': '4bf58dd8d48988d199941735', 'name': 'College Arts Building', 'pluralName': 'College Arts Buildings', 'shortName': 'Arts', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/default_', 'suffix': '.png'}, 'primary': True}]",v-1582940072,False,Room 6036 Sidney Smith Hall 100 St. George Street,Wilcocks Street,43.663008,-79.398793,"[{'label': 'display', 'lat': 43.66300780461368, 'lng': -79.39879315884255}]",1372,M5S3g3,CA,Toronto,ON,Canada,"[Room 6036 Sidney Smith Hall 100 St. George Street (Wilcocks Street), Toronto ON M5S3g3, Canada]",NaN


In [96]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Art Gallery of Ontario,Art Gallery,317 Dundas St W,at Beverley St,43.654003,-79.392922,"[{'label': 'display', 'lat': 43.654002860337386, 'lng': -79.39292172707437}]",460,M5T 1G4,CA,Toronto,ON,Canada,"[317 Dundas St W (at Beverley St), Toronto ON M5T 1G4, Canada]",4ad4c05ef964a520daf620e3
1,Ontario College of Art and Design University (OCADU),University,100 McCaul St,at Dundas St W,43.652803,-79.391074,"[{'label': 'display', 'lat': 43.65280251171013, 'lng': -79.3910743699992}]",337,M5T 1W1,CA,Toronto,ON,Canada,"[100 McCaul St (at Dundas St W), Toronto ON M5T 1W1, Canada]",4ad4c064f964a52065f820e3
2,Aboveground Art Supplies,Arts & Crafts Store,74 McCaul St,Dundas St.,43.652646,-79.390925,"[{'label': 'display', 'lat': 43.652645648070006, 'lng': -79.3909251764317}]",333,M5T 3K2,CA,Toronto,ON,Canada,"[74 McCaul St (Dundas St.), Toronto ON M5T 3K2, Canada]",4adf3c01f964a5208f7821e3
3,CMU College of Makeup Art & Design,School,110 Lombard St.,NaN,43.652516,-79.373252,"[{'label': 'display', 'lat': 43.65251631855119, 'lng': -79.37325153352174}]",1135,M5C 1M3,CA,Toronto,ON,Canada,"[110 Lombard St., Toronto ON M5C 1M3, Canada]",4f64f2c0e4b0c7552d4dbfbf
4,Department of Art,College Arts Building,Room 6036 Sidney Smith Hall 100 St. George Street,Wilcocks Street,43.663008,-79.398793,"[{'label': 'display', 'lat': 43.66300780461368, 'lng': -79.39879315884255}]",1372,M5S3g3,CA,Toronto,ON,Canada,"[Room 6036 Sidney Smith Hall 100 St. George Street (Wilcocks Street), Toronto ON M5S3g3, Canada]",4f0b5bfae4b06fab1c0478f2


In [97]:
#Names
dataframe_filtered.name.head()

0    Art Gallery of Ontario                              
1    Ontario College of Art and Design University (OCADU)
2    Aboveground Art Supplies                            
3    CMU College of Makeup Art & Design                  
4    Department of Art                                   
Name: name, dtype: object

In [98]:
#29 Venues within radius
#Visualization
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Center of Toronto
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Toronto Centre',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [99]:
#Explore a given venue
#https://api.foursquare.com/v2/venues/VENUE_ID?client_id=CLIENT_ID&client_secret=CLIENT_SECRET&v=VERSION

venue_id = '4ad4c064f964a52065f820e3' #Ontario College of Art and Design
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4ad4c064f964a52065f820e3?client_id=2XTXUDPQTGZFAHBYZ1UKI4PNDRO0WYTPVX2BJMYV32ANXOPX&client_secret=MKH4FVRUVQNDC3MAQE350E2ACWGRSL311B5YMMLTXLBQ5CIZ&v=20200201'

In [100]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'venueRatingBlacklisted', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'description', 'page', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4ad4c064f964a52065f820e3',
 'name': 'Ontario College of Art and Design University (OCADU)',
 'contact': {'phone': '4169776000',
  'formattedPhone': '(416) 977-6000',
  'twitter': 'ocad'},
 'location': {'address': '100 McCaul St',
  'crossStreet': 'at Dundas St W',
  'lat': 43.65280251171013,
  'lng': -79.3910743699992,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.65280251171013,
    'lng': -79.3910743699992}],
  'postalCode': 'M5T 1W1',
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['100 McCaul St (at Dundas St W)',
   'Toronto ON M5T 1W1',
   'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/ontario-college-of-art-and-design-university-ocadu/4ad4c064f964a52065f820e3',
 'categories': [{'id': '4bf58dd8d48988d1ae941735',
   'name': 'University',
   'pluralName': 'Universities',
   'shortName': 'University',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/education/default_',
    'suffix': '.png'},
  

In [101]:
#Rating
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.3


In [102]:
#Art Gallery of Ontario

venue_id = '4ad4c05ef964a520daf620e3' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()

In [103]:
#Rating
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

9.2


In [104]:
#Get the number of tips   
result['response']['venue']['tips']['count']

200

In [105]:
#Create URL and send GET request
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)
url

'https://api.foursquare.com/v2/venues/4ad4c05ef964a520daf620e3/tips?client_id=2XTXUDPQTGZFAHBYZ1UKI4PNDRO0WYTPVX2BJMYV32ANXOPX&client_secret=MKH4FVRUVQNDC3MAQE350E2ACWGRSL311B5YMMLTXLBQ5CIZ&v=20200201&limit=15'

In [106]:
#Json
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e59bf31882fc700280e9948'},
 'response': {'tips': {'count': 200,
   'items': [{'id': '4f05d2189adf79d87398ecf8',
     'createdAt': 1325781528,
     'text': "Did you know that the AGO's spiral staircase, designed by Frank Gehry, is purposefully narrow so that people could bump into each other and fall in love. Ahhh!",
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/4f05d2189adf79d87398ecf8',
     'lang': 'en',
     'likes': {'count': 25,
      'groups': [{'type': 'others', 'count': 25, 'items': []}],
      'summary': '25 likes'},
     'logView': True,
     'editedAt': 1372872636,
     'agreeCount': 45,
     'disagreeCount': 0,
     'lastVoteText': 'Upvoted Jan 25',
     'lastUpvoteTimestamp': 1579982967,
     'todo': {'count': 1},
     'user': {'id': '13980167',
      'firstName': 'Art Gallery of Ontario',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/13980167-31YTZ5T0LC4GBK33.gif'},
      't

In [107]:
#Get tips and list of associated features
tips = results['response']['tips']['items']
tips

[{'id': '4f05d2189adf79d87398ecf8',
  'createdAt': 1325781528,
  'text': "Did you know that the AGO's spiral staircase, designed by Frank Gehry, is purposefully narrow so that people could bump into each other and fall in love. Ahhh!",
  'type': 'user',
  'canonicalUrl': 'https://foursquare.com/item/4f05d2189adf79d87398ecf8',
  'lang': 'en',
  'likes': {'count': 25,
   'groups': [{'type': 'others', 'count': 25, 'items': []}],
   'summary': '25 likes'},
  'logView': True,
  'editedAt': 1372872636,
  'agreeCount': 45,
  'disagreeCount': 0,
  'lastVoteText': 'Upvoted Jan 25',
  'lastUpvoteTimestamp': 1579982967,
  'todo': {'count': 1},
  'user': {'id': '13980167',
   'firstName': 'Art Gallery of Ontario',
   'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
    'suffix': '/13980167-31YTZ5T0LC4GBK33.gif'},
   'type': 'page'}}]

In [108]:
#first tip, dict keys
tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'editedAt', 'agreeCount', 'disagreeCount', 'lastVoteText', 'lastUpvoteTimestamp', 'todo', 'user'])

In [109]:
tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'editedAt', 'agreeCount', 'disagreeCount', 'lastVoteText', 'lastUpvoteTimestamp', 'todo', 'user'])

In [110]:
#Format column width and display all tips
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

##Format column width and display all tips
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

C:\Users\hardo\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"Did you know that the AGO's spiral staircase, designed by Frank Gehry, is purposefully narrow so that people could bump into each other and fall in love. Ahhh!",45,0,4f05d2189adf79d87398ecf8,Art Gallery of Ontario,NaN,NaN,13980167


In [77]:
#User of the last comment

#https://foursquare.com/developers/explore
#https://api.foursquare.com/v2/users/13980167?client_id=2XTXUDPQTGZFAHBYZ1UKI4PNDRO0WYTPVX2BJMYV32ANXOPX&client_secret=MKH4FVRUVQNDC3MAQE350E2ACWGRSL311B5YMMLTXLBQ5CIZ&v=20200201

'https://api.foursquare.com/v2/users/13980167?client_id=2XTXUDPQTGZFAHBYZ1UKI4PNDRO0WYTPVX2BJMYV32ANXOPX&client_secret=MKH4FVRUVQNDC3MAQE350E2ACWGRSL311B5YMMLTXLBQ5CIZ&v=20200201&limit=30'

In [111]:
#Invalidid credentials for users
url = 'https://api.foursquare.com/v2/users/13980167?client_id=2XTXUDPQTGZFAHBYZ1UKI4PNDRO0WYTPVX2BJMYV32ANXOPX&client_secret=MKH4FVRUVQNDC3MAQE350E2ACWGRSL311B5YMMLTXLBQ5CIZ&v=20200201'

results = requests.get(url).json()
results

{'meta': {'code': 401,
  'errorType': 'invalid_auth',
  'errorDetail': 'Missing oauth_token. See https://developer.foursquare.com/docs/api/configuration/authentication for details.',
  'requestId': '5e59bfe9be61c9001be381f0'},
 'response': {}}

In [120]:
#Get user's tips
# define tips URL
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

# send GET request and get user's tips
results = requests.get(url).json()
tips = results['response']['tips']['items']

# format column width
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

# filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display user's tips
tips_filtered

,text,agreeCount,disagreeCount,id
0,"Did you know that the AGO's spiral staircase, designed by Frank Gehry, is purposefully narrow so that people could bump into each other and fall in love. Ahhh!",45,0,4f05d2189adf79d87398ecf8
1,"Situated in the Grange (the oldest part of the Gallery), our Members' Lounge is a great spot to rest your feet. Cosy yet refined and with lovely views of the park.",1,0,4f05c3c299116f05b5e61e4c
2,The cheese and ham croissants could quite possibly be the best in the city. Just saying...,2,0,4f05b9ca46909da7013500ac
3,From 3pm on weekdays we have a range of exciting free youth classes including DJ lessons and hip-hop dance. Check out our youth Facebook page for more details!,1,0,4f05b8a229c22bbeff0f9e5f


In [121]:
#Greatest agree counts
#No user data, invalid credentials

#Let's get the venue for the tip with the greatest number of agree counts
tip_id = '4f05d2189adf79d87398ecf8' # tip id

# define URL
url = 'http://api.foursquare.com/v2/tips/{}?client_id={}&client_secret={}&v={}'.format(tip_id, CLIENT_ID, CLIENT_SECRET, VERSION)

# send GET Request and examine results
result = requests.get(url).json()
print(result['response']['tip']['venue']['name'])
print(result['response']['tip']['venue']['location'])

#Get User's friends
user_friends = json_normalize(user_data['friends']['groups'][0]['items']) #No user data
user_friends

Art Gallery of Ontario
{'address': '317 Dundas St W', 'crossStreet': 'at Beverley St', 'lat': 43.654002860337386, 'lng': -79.39292172707437, 'labeledLatLngs': [{'label': 'display', 'lat': 43.654002860337386, 'lng': -79.39292172707437}], 'postalCode': 'M5T 1G4', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['317 Dundas St W (at Beverley St)', 'Toronto ON M5T 1G4', 'Canada']}


NameError: name 'user_data' is not defined

In [131]:
#Explore a location in Toronto

latitude = 43.662852
longitude = -79.396450

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e59c875b57e88001b5e3967'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 241,
  'suggestedBounds': {'ne': {'lat': 43.68985202700003,
    'lng': -79.35919667039902},
   'sw': {'lat': 43.63585197299997, 'lng': -79.43370332960099}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b9d206bf964a520e69136e3',
       'name': "Queen's Park",
       'location': {'address': 'University Ave.',
        'crossStreet': 'at Wellesley Ave.',
        'lat': 43.66394609897775,
        'lng': -79.39217952520835,
        'labeledLatLngs': [{'label': 'display',
 

In [132]:
#First Item
import requests
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4b9d206bf964a520e69136e3',
  'name': "Queen's Park",
  'location': {'address': 'University Ave.',
   'crossStreet': 'at Wellesley Ave.',
   'lat': 43.66394609897775,
   'lng': -79.39217952520835,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.66394609897775,
     'lng': -79.39217952520835}],
   'distance': 364,
   'postalCode': 'M5R 2E8',
   'cc': 'CA',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['University Ave. (at Wellesley Ave.)',
    'Toronto ON M5R 2E8',
    'Canada']},
  'categories': [{'id': '4bf58dd8d48988d163941735',
    'name': 'Park',
    'pluralName': 'Parks',
    'shortName': 'Park',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 're

In [133]:
#Process JSON and convert it to a clean dataframe
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Queen's Park,Park,University Ave.,at Wellesley Ave.,43.663946,-79.392180,"[{'label': 'display', 'lat': 43.66394609897775, 'lng': -79.39217952520835}]",364,M5R 2E8,CA,Toronto,ON,Canada,"[University Ave. (at Wellesley Ave.), Toronto ON M5R 2E8, Canada]",NaN,4b9d206bf964a520e69136e3
1,Philosopher's Walk,Park,NaN,University of Toronto,43.666894,-79.395597,"[{'label': 'display', 'lat': 43.666893747498605, 'lng': -79.39559698104858}]",455,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,4ba79cd4f964a52060a239e3
2,Koerner Hall,Concert Hall,273 Bloor St. W,at Avenue Rd.,43.667983,-79.395962,"[{'label': 'display', 'lat': 43.667983280307865, 'lng': -79.3959619437764}]",572,M5S 1W2,CA,Toronto,ON,Canada,"[273 Bloor St. W (at Avenue Rd.), Toronto ON M5S 1W2, Canada]",University of Toronto,4adf787cf964a520e57a21e3
3,Rasa,Restaurant,196 Robert Street,NaN,43.662757,-79.403988,"[{'label': 'display', 'lat': 43.662756751275445, 'lng': -79.40398803188654}]",607,M5S 2K8,CA,Toronto,ON,Canada,"[196 Robert Street, Toronto ON M5S 2K8, Canada]",Downtown Toronto,527d450111d25050de4ea0d8
4,Royal Ontario Museum,Museum,100 Queen's Pk,at Bloor St. W.,43.668367,-79.394813,"[{'label': 'display', 'lat': 43.668367065616046, 'lng': -79.3948129405877}]",627,M5S 2C6,CA,Toronto,ON,Canada,"[100 Queen's Pk (at Bloor St. W.), Toronto ON M5S 2C6, Canada]",University,4ad4c05ef964a520d9f620e3
5,ROM Museum Store,Gift Shop,100 Queen's Park,inside Royal Ontario Museum,43.668514,-79.394879,"[{'label': 'display', 'lat': 43.66851433563614, 'lng': -79.39487921364}]",642,M5S 2C6,CA,Toronto,ON,Canada,"[100 Queen's Park (inside Royal Ontario Museum), Toronto ON M5S 2C6, Canada]",NaN,4fce7df0e4b0d498825b14d2
6,Her Father's Cider Bar + Kitchen,Beer Bar,119 Harbord Street,at Major St,43.662448,-79.404703,"[{'label': 'display', 'lat': 43.66244824890102, 'lng': -79.40470324675829}]",666,M5S 1G7,CA,Toronto,ON,Canada,"[119 Harbord Street (at Major St), Toronto ON M5S 1G7, Canada]",Downtown Toronto,5707bcb9498e709b544e20b2
7,Royal Conservatory of Music,Music School,273 Bloor St W,NaN,43.668228,-79.396311,"[{'label': 'display', 'lat': 43.66822758218325, 'lng': -79.39631143267306}]",598,M5S 1W2,CA,Toronto,ON,Canada,"[273 Bloor St W, Toronto ON M5S 1W2, Canada]",NaN,4b31677bf964a520530625e3
8,Fresh on Bloor,Vegetarian / Vegan Restaurant,326 Bloor St. W,NaN,43.666755,-79.403491,"[{'label': 'display', 'lat': 43.66675488472059, 'lng': -79.40349130034014}]",714,M5S 1W5,CA,Toronto,ON,Canada,"[326 Bloor St. W, Toronto ON M5S 1W5, Canada]",NaN,4df15e3c1f6e818dadf7afc0
9,The Planet Traveler,Hostel,357 College Street,NaN,43.657202,-79.403568,"[{'label': 'display', 'lat': 43.657202, 'lng': -79.403568}]",850,M5T 1S5,CA,Toronto,ON,Canada,"[357 College Street, Toronto ON M5T 1S5, Canada]",NaN,4d947c8c1646a35df82237a3


In [134]:
#Visualization

venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Location',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

In [135]:
#Explore trading Venues

# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e59c75c40a7ea001b1fa76f'},
 'response': {'venues': []}}

In [136]:
#Check if venues
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)
    
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

In [137]:
#Visualization
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)
        
# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'